In [20]:
def SmallParsimony(Tree,Parent, Nodes,Character,n,Text):
    
    #Text = {node:'' for node in Nodes}
    #for each node v in tree T
    ####Initilize 
    #Text = {node:'' for node in Nodes}
    s = dict()
    for k in alphabet:
        s[k] = {node:float('inf') for node in Nodes}
    
    Tag = dict()
    for node in Nodes:
        Tag[node] = 0
    Ripe=dict()
    for parent,children in Tree.items():
        for child in children:
            if parent in Ripe.keys():
                Ripe[parent] += 1-Tag[child]
            else:
                Ripe[parent] = 1-Tag[child]
    
    
    for v in Nodes:
        #Tag[v] = 0
        #if v is a leaf
        if v < n:
            Tag[v] = 1
            #####The parent node -1, 0 for ripe
            Ripe[Parent[v]] -= 1
            #for each symbol k in the alphabet
            for k in alphabet:
                if Character[v] == k:
                    s[k][v] = 0
    
    #while there exist ripe nodes in T
    ripeNodes = [key for key,val in Ripe.items() if val==0]
    while(ripeNodes):
        #v ← a ripe node in T
        for v in ripeNodes:
            Tag[v] =  1
            #Nodes.remove(v)
            if v in Parent.keys():
                Ripe[Parent[v]] -= 1
            del Ripe[v]
            #for each symbol k in the alphabet
            Daughter,Son = Tree[v]
            for k in alphabet:
                #sk(v) ← minimumall symbols i {si(Daughter(v))+αi,k} + minimumall symbols j {sj(Son(v))+αj,k}
                Dau_score,Dau_car = minimumall(Daughter,k,s)
                Son_score,Son_car = minimumall(Son,k,s)
                s[k][v] = Dau_score+Son_score
        ripeNodes = [key for key,val in Ripe.items() if val==0]
    
    minScore = float('inf')
    minSymbol = ''
    for k in alphabet:
        if(minScore>s[k][v]):
            minScore= s[k][v]
            minSymbol = k
    Text[v] += minSymbol
    for node in Tree[v]:
        minimumOver(minSymbol,node,s,Text)
        
    #return minimum over all symbols k {sk(v)}
    return s,Text
    

In [21]:
def minimumOver(k,v, s,Text):
    if (v<n):
        Text[v] += Character[v]
        return None
        
    minScore = float('inf')
    minSymbol = ''
    for i,val in s.items():
        alpha = 1
        if (i==k):
            alpha=0
        score = val[v]+alpha
        if(minScore>score):
            minSymbol = i
            minScore=score
    Text[v] += minSymbol
    for node in Tree[v]:
        minimumOver(minSymbol,node,s,Text)
    return None
    

In [22]:
def minimumall(child,k,s):
    minSymbol = ''
    minScore = float('inf')
    for i,val in s.items():
        alpha = 1
        if (i==k):
            alpha=0
            
        score = val[child]+alpha
        if(minScore>score):
            minScore = score
            minSymbol = i
    return minScore,minSymbol

In [80]:
def get_rooted_tree(n):
    Nodes = [i for i in range(2*n-2,-1,-1)]
    Tree={}
    Parent={}
    for i,node in enumerate(Nodes):
        
        if (node<n):
            break
        Tree[node] = [Nodes[2*i+1],Nodes[2*i+2]]
        Parent[Nodes[2*i+1]]= node
        Parent[Nodes[2*i+2]]= node  
        
    return Tree,Parent,Nodes
    
    

In [85]:
alphabet = ['A','T','C','G']
edges = [(4,0),(4,1),(5,2),(5,3),(4,5)]
#labels = ['CAAATCCC', 'ATTGCGAC', 'CTGCGCTG', 'ATGGACGA']
labels = [ 'TCGGCCAA','CCTGGCTG','CACAGGAT','TGAGTACC']
# compute tree and reverse-tree from edges
#Tree = {}
#Parent = {}
Nodes = set()
n =4

for edge in edges:
    node = edge[0]
    child = edge[1]
    Nodes.add(node)
    Nodes.add(child)
    #Tree.setdefault(node,[]).append(child)
    #Parent[child] = node


Tree,Parent,Nodes=get_rooted_tree(n)
Text = {node:'' for node in Nodes}

In [86]:
for i in range(len(labels[0])):
    Character = dict()
    for idx,label in enumerate(labels):
        Character[idx] = label[i]
    s,Text = SmallParsimony(Tree,Parent, Nodes,Character,n,Text)

In [87]:
def hamming(s1,s2):
    assert len(s1) == len(s2)
    d = 0
    for i in range(len(s1)):
        if s1[i]!=s2[i]:
            d+=1
    return d

In [88]:
score = 0
for parent,children in edges:
    par = Text[parent]
    chi = Text[children]
    dist = hamming(par,chi)
    print('%s->%s:%d'%(par,chi,dist))
    print('%s->%s:%d'%(chi,par,dist))
    score += dist
print(score)

TCAGGCAA->TCGGCCAA:2
TCGGCCAA->TCAGGCAA:2
TCAGGCAA->CCTGGCTG:4
CCTGGCTG->TCAGGCAA:4
TAAGGAAA->CACAGGAT:5
CACAGGAT->TAAGGAAA:5
TAAGGAAA->TGAGTACC:4
TGAGTACC->TAAGGAAA:4
TCAGGCAA->TAAGGAAA:2
TAAGGAAA->TCAGGCAA:2
17
